In [1]:
#导入库
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torchvision #加载图片
from torchvision import transforms #图片变换

from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
 
import numpy as np
import matplotlib.pyplot as plt #绘图
import os
import glob
from PIL import Image
import random

from preprocess import Process
from transformer import *

In [2]:
seed = 26
def get_random_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
get_random_seed(seed)

In [3]:
#独热编码
def one_hot(x,class_count=10):
    return torch.eye(class_count)[x,:]

In [4]:
series = []
items = 3
step = 3
subject_num = 0

for num in range(2, 66):
    try:
        series += Process(num).prepro(1024, step, items)
        subject_num += 1
    except:
        print(f'subject {num} abandoned')

subject 13 abandoned
subject 16 abandoned
subject 17 abandoned
subject 18 abandoned
subject 20 abandoned
subject 26 abandoned


/home/dhz/anaconda3/lib/python3.9/site-packages/scipy/interpolate/fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/home/dhz/anaconda3/lib/python3.9/site-packages/numpy/ma/core.py:5244: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
/home/dhz/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


subject 42 abandoned
subject 47 abandoned
subject 48 abandoned
subject 50 abandoned


In [5]:
class MyDataset(Dataset):
    def __init__(self, series, items, subject_num):
        self.series = series
        self.codes = []
        self.subject_num = subject_num
        self.labels = np.zeros((len(self.series), self.subject_num), dtype='double')
        for i in range(self.subject_num):
            for j in range(items):
                self.labels[i * 3 + j][i] = 1.0            

    # need to overload
    def __len__(self):
        return len(self.series)

    # need to overload
    def __getitem__(self, idx):
        return torch.tensor(self.series[idx]), torch.tensor(self.labels[idx])

In [6]:
dataset = MyDataset(series, items=items, subject_num=subject_num)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=16)

In [7]:
#定义生成器
class Generator(nn.Module):
    def __init__(self, subject_num):
        super(Generator,self).__init__()
        self.linear1 = nn.Linear(100,512)
        self.bn1=nn.BatchNorm1d(512)
        self.subject_num = subject_num
        self.linear2 = nn.Linear(subject_num,512)
        self.bn2=nn.BatchNorm1d(512)
        
        self.deconv1 = nn.Conv1d(1, 3, kernel_size=2, padding='same')
        self.bn3=nn.BatchNorm1d(3)
        self.deconv2 = nn.Conv1d(3, 6, kernel_size=2, padding='same')
        self.bn4=nn.BatchNorm1d(6)
        self.deconv3 = nn.Conv1d(6, 1, kernel_size=2, padding='same')
        
    def forward(self,x1,x2):
        x1=F.relu(self.linear1(x1.to(torch.float64)))
        x1=self.bn1(x1)
        x2=F.relu(self.linear2(x2))
        x2=self.bn2(x2)
        x=torch.cat([x1,x2],axis=1)
        x=F.relu(self.deconv1(torch.reshape(x, (x.size(0), 1, x.size(1)))))
        x=self.bn3(x)
        x=F.relu(self.deconv2(x))
        x=self.bn4(x)
        x=torch.tanh(self.deconv3(x))
        return x

In [8]:
#定义判别器
#输入：1，28，28图片和长度为10的condition
class Discriminator(nn.Module):
    def __init__(self, subject_num):
        super(Discriminator,self).__init__()
        self.subject_num = subject_num
        self.linear = nn.Linear(self.subject_num,1024)
        self.conv1 = nn.Conv1d(1,32,kernel_size=2,padding='same')
        self.conv2 = nn.Conv1d(32,128,kernel_size=2,padding='same')
        self.bn = nn.BatchNorm1d(128)
        self.fc = nn.Linear(2048,1)
    def forward(self,x1,x2): #x1代表label,x2代表image
        x1=F.leaky_relu(self.linear(x1))
        x=torch.cat([x1,x2],axis=1)            
        x= F.dropout1d(F.leaky_relu(self.conv1(torch.reshape(x, (x.size(0), 1, x.size(1))))))
        x= F.dropout1d(F.leaky_relu(self.conv2(x)))
        x = self.bn(x)
        x = torch.sigmoid(self.fc(x))
        return x

In [9]:
class ContinuityLoss(nn.Module):
    def __init__(self, weight=1.0):
        super(ContinuityLoss, self).__init__()
        self.weight = weight
        
    def forward(self, predictions):
        diff = predictions[:][1:] - predictions[:][:-1]
        loss = torch.mean(torch.abs(diff))
        return loss * self.weight

In [10]:
class ShapeLoss(nn.Module):
    def __init__(self, weight=1.0):
        super(ShapeLoss, self).__init__()
        self.weight = weight
        
    def forward(self, predictions, labels):
        loss = torch.mean(torch.abs(predictions - labels))
        return loss * self.weight

In [11]:
class MaxValueLoss(nn.Module):
    def __init__(self, weight=1.0):
        super(MaxValueLoss, self).__init__()
        self.weight = weight
        self.MCE = nn.CrossEntropyLoss()
    def forward(self, input1):
        input2 = torch.zeros_like(input1)
        types = torch.argmax(input1, dim = 0, keepdim=False)
        for i in range(input1.size(0)):
            input2[i][types[i]] = 1
        loss = self.MCE(input1, input2)
        return loss * self.weight

In [12]:
class ClassLoss(nn.Module):
    def __init__(self, weight=1.0):
        super(ClassLoss, self).__init__()
        self.weight = weight
    def forward(self, input1, input2):
        diff = input1 - input2
        loss = torch.mean(torch.abs(diff))
        return loss * self.weight

In [13]:
class ClassLoss(nn.Module):
    def __init__(self, weight=1.0):
        super(ClassLoss, self).__init__()
        self.weight = weight
        self.MCE = nn.CrossEntropyLoss()
    def forward(self, input1, input2):
        loss = self.MCE(input1, input2)
        return loss * self.weight

In [14]:
#设备的配置
device='cuda' if torch.cuda.is_available() else 'cpu'
#初化生成器和判别器把他们放到相应的设备上
gen = Generator(subject_num).to(device)
gen = gen.double()
dis = Discriminator(subject_num).to(device)
dis = dis.double()
cls = torch.load("classification").to(device)
cls = cls.double()
#交叉熵损失函数
loss_fn = torch.nn.BCELoss()
continue_loss_weight = 0.8
continue_loss = ShapeLoss(weight=continue_loss_weight)
class_loss_weight = 0.2
class_loss = ClassLoss(weight=class_loss_weight)
#训练器的优化器
discriminator_rate = 1e-5
generator_rate = 1e-4
d_optimizer = torch.optim.Adam(dis.parameters(),lr=discriminator_rate)
#训练生成器的优化器
g_optimizer = torch.optim.Adam(gen.parameters(),lr=generator_rate)

In [15]:
#设置生成绘图图片的随机张量，这里可视化16张图片
#生成16个长度为100的随机正态分布张量
noise_seed = torch.randn(16,100,device=device)
label_seed = torch.randint(0,subject_num,size=(16,))
label_seed_onehot = one_hot(label_seed, class_count=subject_num).to(device)
 
D_loss = [] #记录训练过程中判别器的损失
G_loss = [] #记录训练过程中生成器的损失
S_loss = []
C_loss = []
accs = []
epoch_num = 4000

In [ ]:
#训练循环
for epoch in range(epoch_num):
    #初始化损失值
    D_epoch_loss = 0
    G_epoch_loss = 0
    C_epoch_loss = 0
    S_epoch_loss = 0
    acc_num = 0
    item_num = 0
    count = len(dataloader.dataset) #返回批次数
    #对数据集进行迭代
    for step,(img,label) in enumerate(dataloader):
        img =img.to(device) #把数据放到设备上
        label = label.to(device)
        size = img.shape[0] #img的第一位是size,获取批次的大小
        random_seed = torch.randn(size,100,device=device)
        item_num += size
        
        #判别器训练(真实图片的损失和生成图片的损失),损失的构建和优化
        d_optimizer.zero_grad()#梯度归零
        #判别器对于真实图片产生的损失
        real_output = dis(label,img) #判别器输入真实的图片，real_output对真实图片的预测结果
        d_real_loss = loss_fn(real_output,
                              torch.ones_like(real_output,device=device)
                              )
        d_real_loss.backward()#计算梯度
        
        #在生成器上去计算生成器的损失，优化目标是判别器上的参数
        generated_img = gen(random_seed,label) #得到生成的图片
        
        #因为优化目标是判别器，所以对生成器上的优化目标进行截断
        generated_img = torch.reshape(generated_img, (generated_img.size(0), generated_img.size(2)))
        fake_output = dis(label,generated_img.detach()) #判别器输入生成的图片，fake_output对生成图片的预测;detach会截断梯度，梯度就不会再传递到gen模型中了
        #判别器在生成图像上产生的损失
        d_fake_loss = loss_fn(fake_output,
                              torch.zeros_like(fake_output,device=device)
                              )
        d_fake_loss.backward()
        #判别器损失
        disc_loss = d_real_loss + d_fake_loss
        #判别器优化
        d_optimizer.step()
        
        
        #生成器上损失的构建和优化
        g_optimizer.zero_grad() #先将生成器上的梯度置零
        fake_output = dis(label,generated_img)
        class_output = cls(torch.reshape(generated_img, (generated_img.size(0), 1, generated_img.size(1))))
        cls_loss = class_loss(class_output, label)
        series_loss = continue_loss(generated_img, img)
        for i in range(size):
            if class_output[i][torch.argmax(label[i])] >= 0.5:
                acc_num += 1
        gen_loss = loss_fn(fake_output,
                              torch.ones_like(fake_output,device=device)
                          ) + series_loss + cls_loss
        gen_loss.backward()
        g_optimizer.step()
        #累计每一个批次的loss
        with torch.no_grad():
            D_epoch_loss +=disc_loss
            G_epoch_loss +=gen_loss
            S_epoch_loss +=series_loss
            C_epoch_loss +=cls_loss
    #求平均损失
    with torch.no_grad():
        D_epoch_loss /=count
        G_epoch_loss /=count
        S_epoch_loss /=(count * continue_loss_weight)
        C_epoch_loss /=(count * class_loss_weight)
        acc = acc_num / item_num
        D_loss.append(D_epoch_loss)
        G_loss.append(G_epoch_loss)
        S_loss.append(S_epoch_loss)
        C_loss.append(C_epoch_loss)
        accs.append(acc)
        print(f"Epoch [{epoch + 1}/{epoch_num}], "
              f"Discriminator Loss: {D_epoch_loss:.4f}, "
              f"Generator Loss: {G_epoch_loss:.4f}, "
              f"Series Loss: {S_epoch_loss:.4f}, "
              f"Class Loss: {C_epoch_loss:.4f}, "
              f"Accuracy: {acc:.4f}")

/home/dhz/anaconda3/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch [1/4000], Discriminator Loss: 0.0963, Generator Loss: 0.1354, Series Loss: 0.0404, Class Loss: 0.2715, Accuracy: 0.1358
Epoch [2/4000], Discriminator Loss: 0.0960, Generator Loss: 0.1346, Series Loss: 0.0399, Class Loss: 0.2713, Accuracy: 0.1481
Epoch [3/4000], Discriminator Loss: 0.0957, Generator Loss: 0.1339, Series Loss: 0.0394, Class Loss: 0.2709, Accuracy: 0.1420
Epoch [4/4000], Discriminator Loss: 0.0957, Generator Loss: 0.1335, Series Loss: 0.0390, Class Loss: 0.2708, Accuracy: 0.1481
Epoch [5/4000], Discriminator Loss: 0.0953, Generator Loss: 0.1329, Series Loss: 0.0386, Class Loss: 0.2706, Accuracy: 0.2037
Epoch [6/4000], Discriminator Loss: 0.0953, Generator Loss: 0.1324, Series Loss: 0.0382, Class Loss: 0.2706, Accuracy: 0.2284
Epoch [7/4000], Discriminator Loss: 0.0952, Generator Loss: 0.1320, Series Loss: 0.0378, Class Loss: 0.2702, Accuracy: 0.2037
Epoch [8/4000], Discriminator Loss: 0.0946, Generator Loss: 0.1316, Series Loss: 0.0375, Class Loss: 0.2701, Accuracy:

Epoch [66/4000], Discriminator Loss: 0.0904, Generator Loss: 0.1252, Series Loss: 0.0281, Class Loss: 0.2671, Accuracy: 0.5062
Epoch [67/4000], Discriminator Loss: 0.0908, Generator Loss: 0.1249, Series Loss: 0.0280, Class Loss: 0.2670, Accuracy: 0.5000
Epoch [68/4000], Discriminator Loss: 0.0903, Generator Loss: 0.1252, Series Loss: 0.0279, Class Loss: 0.2670, Accuracy: 0.5123
Epoch [69/4000], Discriminator Loss: 0.0907, Generator Loss: 0.1250, Series Loss: 0.0279, Class Loss: 0.2670, Accuracy: 0.4815
Epoch [70/4000], Discriminator Loss: 0.0905, Generator Loss: 0.1250, Series Loss: 0.0277, Class Loss: 0.2668, Accuracy: 0.5000
Epoch [71/4000], Discriminator Loss: 0.0903, Generator Loss: 0.1250, Series Loss: 0.0277, Class Loss: 0.2668, Accuracy: 0.5309
Epoch [72/4000], Discriminator Loss: 0.0902, Generator Loss: 0.1249, Series Loss: 0.0276, Class Loss: 0.2668, Accuracy: 0.5123
Epoch [73/4000], Discriminator Loss: 0.0904, Generator Loss: 0.1247, Series Loss: 0.0275, Class Loss: 0.2667, A

Epoch [131/4000], Discriminator Loss: 0.0876, Generator Loss: 0.1237, Series Loss: 0.0257, Class Loss: 0.2665, Accuracy: 0.5370
Epoch [132/4000], Discriminator Loss: 0.0876, Generator Loss: 0.1246, Series Loss: 0.0257, Class Loss: 0.2666, Accuracy: 0.5679
Epoch [133/4000], Discriminator Loss: 0.0881, Generator Loss: 0.1245, Series Loss: 0.0257, Class Loss: 0.2669, Accuracy: 0.4691
Epoch [134/4000], Discriminator Loss: 0.0878, Generator Loss: 0.1246, Series Loss: 0.0257, Class Loss: 0.2665, Accuracy: 0.5494
Epoch [135/4000], Discriminator Loss: 0.0876, Generator Loss: 0.1240, Series Loss: 0.0257, Class Loss: 0.2667, Accuracy: 0.5432
Epoch [136/4000], Discriminator Loss: 0.0877, Generator Loss: 0.1241, Series Loss: 0.0258, Class Loss: 0.2664, Accuracy: 0.5494
Epoch [137/4000], Discriminator Loss: 0.0877, Generator Loss: 0.1240, Series Loss: 0.0257, Class Loss: 0.2665, Accuracy: 0.5741
Epoch [138/4000], Discriminator Loss: 0.0877, Generator Loss: 0.1236, Series Loss: 0.0257, Class Loss: 0

In [ ]:
g_curve = [i.cpu() for i in G_loss]
plt.plot(g_curve)

In [ ]:
d_curve = [i.cpu() for i in D_loss]
plt.plot(d_curve)

In [ ]:
s_curve = [i.cpu() for i in S_loss]
plt.plot(s_curve)

In [ ]:
c_curve = [i.cpu() for i in C_loss]
plt.plot(c_curve)

In [ ]:
a_curve = [i for i in accs]
plt.plot(a_curve)

In [ ]:
output = generated_img.cpu().detach()
# output = torch.reshape(output, (output.size(1), output.size(0)))

In [ ]:
plt.plot(output.numpy()[1])

In [ ]:
def transpose_list(my_list):
    transposed_list = list(zip(*my_list))
    for i in range(len(transposed_list)):
        transposed_list[i] = list(transposed_list[i])
    return transposed_list

In [ ]:
transposed_list = transpose_list(series)

In [ ]:
plt.plot(series[0])

In [ ]:
auth = []
for step,(img,label) in enumerate(dataloader):
        img =img.to(device) #把数据放到设备上
        label = label.to(device)
        size = img.shape[0] #img的第一位是size,获取批次的大小
        random_seed = torch.randn(size,100,device=device)
        
        #在生成器上去计算生成器的损失，优化目标是判别器上的参数
        generated_img = gen(random_seed,label) #得到生成的图片
        generated_img = torch.reshape(generated_img, (generated_img.size(0), generated_img.size(2)))
        class_output = cls(torch.reshape(generated_img, (generated_img.size(0), 1, generated_img.size(1))))
        true_output = cls(torch.reshape(img, (img.size(0), 1, img.size(1))))
        print('fake output:')
        for item in class_output:
            print(torch.max(item))
        print('true output:')
        for item in true_output:
            print(torch.max(item))
        auth.append(class_output)

In [ ]:
auth

In [ ]:
path = 'shapeloss/seed{}'.format(seed) + 'G{}'.format(generator_rate) + 'D{}'.format(discriminator_rate) + 'epoch{}'.format(epoch_num)
isExists=os.path.exists(path)
if not isExists:
    os.mkdir(path)
path = path + '/S{}'.format(continue_loss_weight) + 'C{}'.format(class_loss_weight)
os.mkdir(path)
torch.save(gen, path + '/generator.pt')

In [ ]:
torch.save(dis, path + "/discriminator.pt")

In [ ]:
code = []
for _, (_, _) in enumerate(dataloader):
    random_seed = torch.randn(size,100,device=device)
    code.append(random_seed.cpu().numpy())

In [ ]:
np.array(code)

In [ ]:
np.save(path + '/code.npy', code)

In [ ]:
torch.save(accs, path + '/accs.pt')